In [3]:
#전이학습을 위한 이미지 특성 벡터 생성하는 모델 선택하기 
handle_base_name = "mobilenet_v2_tf2_pre_fv_v4" # @param ['mobilenet_v2_tf2_pre_fv_v4', 'mobilenet_v2_140_224_v2', 'effNet_v2_1k_b0_fv_v2']

#링크 
handle_base_map = {
  "mobilenet_v2_tf2_pre_fv_v4": "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-feature-vector/versions/4",
  "mobilenet_v2_140_224_v2": "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/140-224-feature-vector/versions/2",
  "effNet_v2_1k_b0_fv_v2": "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-feature-vector/versions/2",
}

#인풋 이미지 사이즈 크기 
handle_base_image_size_map = {
  "mobilenet_v2_tf2_pre_fv_v4": 224,
  "mobilenet_v2_140_224_v2": 224,
  "effNet_v2_1k_b0_fv_v2": 224,
}

#아웃풋 FEATURE VECTOR SIZE 
handle_base_output_map ={
  "mobilenet_v2_tf2_pre_fv_v4":1280,
  "mobilenet_v2_140_224_v2": 1792,
  "effNet_v2_1k_b0_fv_v2": 1280  
    
}

model_handle = handle_base_map.get(handle_base_name)
pixels = handle_base_image_size_map.get(handle_base_name, 224) #(2번째 파라미터는 해당 모델이 없을 시, 디폴트값
feature_vector_size = handle_base_output_map.get(handle_base_name)

print(f"Selected model: {handle_base_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}, Output feature vector size {feature_vector_size}")

feature_extractor = hub.KerasLayer(
    model_handle,
    input_shape = IMAGE_SIZE +(3,),
    output_shape = [feature_vector_size],
    trainable = False
)

# 모델 선언
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])





In [ ]:
###############################
#
# 말라리아 malaria
#['parasitized', 'uninfected']
#mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-feature-vector/versions/4
################################

import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tflite_runtime.interpreter as tflite

###########전이학습을 위한 이미지 특성 벡터 생성하는 모델 선택하기########## 
handle_base_name = "effNet_v2_1k_b0_fv_v2" # @param ['mobilenet_v2_tf2_pre_fv_v4', 'mobilenet_v2_140_224_v2', 'effNet_v2_1k_b0_fv_v2']

#링크 
handle_base_map = {
  "mobilenet_v2_tf2_pre_fv_v4": "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-feature-vector/versions/4",
  "mobilenet_v2_140_224_v2": "https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/140-224-feature-vector/versions/2",
  "effNet_v2_1k_b0_fv_v2": "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-feature-vector/versions/2",
}

#인풋 이미지 사이즈 크기 
handle_base_image_size_map = {
  "mobilenet_v2_tf2_pre_fv_v4": 224,
  "mobilenet_v2_140_224_v2": 224,
  "effNet_v2_1k_b0_fv_v2": 224,
}

#아웃풋 FEATURE VECTOR SIZE 
handle_base_output_map ={
  "mobilenet_v2_tf2_pre_fv_v4":1280,
  "mobilenet_v2_140_224_v2": 1792,
  "effNet_v2_1k_b0_fv_v2": 1280  
    
}

model_handle = handle_base_map.get(handle_base_name)
pixels = handle_base_image_size_map.get(handle_base_name, 224) #(2번째 파라미터는 해당 모델이 없을 시, 디폴트값
feature_vector_size = handle_base_output_map.get(handle_base_name)
IMAGE_SIZE = (pixels, pixels)

print(f"Selected model: {handle_base_name} : {model_handle}")
print(f"Input size {IMAGE_SIZE}, Output feature vector size {feature_vector_size}")

feature_extractor = hub.KerasLayer(
    model_handle,
    input_shape = IMAGE_SIZE +(3,),
    output_shape = [feature_vector_size],
    trainable = False
)
BATCH_SIZE = 32

##########데이터세트 설정하기 (모델에 맞게 이미지 조정)##########

#244x 244 사이즈로 리사이즈를 하고, 정규화를 해줍니다.
def format_image(image,label):
    image = tf.image.resize(image,(224,224))/255.0
    return image, label

#훈련, 검증, 테스트 세트로 나눕니다.
(raw_train, raw_validation, raw_test), metadata =tfds.load(
    'malaria',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

print(metadata)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
class_names =metadata.features['label'].names
print(class_names)

#train data 대표 이미지 9개 보여주기 
fig = tfds.show_examples(raw_train, metadata)

train_batches = raw_train.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

##########모델 정의하기#############

model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation ='softmax')
])

model.compile(
    optimizer='adam',
    loss ='sparse_categorical_crossentropy',
    metrics =['accuracy']
)

hist = model.fit(
    train_batches,
    epochs = 5,
    validation_data = validation_batches
)

model.summary()

#트레이닝 결과 그래프 보여주기 
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

#학습된 모델 saved_model로 저장하기 
_SAVED_MODEL = "/content/drive/MyDrive/INTEL_PYTHON/malaria_saved_model"
tf.saved_model.save(model, _SAVED_MODEL)

# tensoflow Lite 파일 생성하기 
converter = tf.lite.TFLiteConverter.from_saved_model(_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT] #Dynamic Range Quantization
tflite_model = converter.convert()
tflite_model_file = '/content/drive/MyDrive/INTEL_PYTHON/malaria_converted_model.tflite'

#with open(…) as f 에서 f는 open(…)함수가 리턴한 file object.
with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)


###########################
## 라즈베리 파이에서 실행할 추론 코드 
###########################

#인터프리터 생성 
interpreter = tflite.Interpreter(model_path='/content/drive/MyDrive/INTEL_PYTHON/malaria_converted_model.tflite')

#텐서 할당
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

input_details = interpreter.get_input_details()[0]
print('input: ', input_details)
output_details = interpreter.get_output_details()[0]
print('output: ', output_details)



predictions =[]

print(input_index)
print(output_index)

#추론하기 
predictions =[]
test_labels, test_imgs = [],[]
for img, label in test_batches.take(100):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    test_labels.append(label.numpy()[0])
    test_imgs.append(img)


#추론 결과 점수로 확인하기 
score  = 0
for item in range(0,100):
    prediction = np.argmax(predictions[item])
    label = test_labels[item]
    if prediction == label:
        score = score +1

print("100개 중 맞은 예측 수: "+ str(score))

def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    img = tf.squeeze(img) 
    plt.imshow(img)

    predicted_label = np.argmax(predictions_array[index])
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                    100*np.max(predictions_array),
                                    class_names[true_label]),
                                    color=color)


for index in range(0,30):

    if index%2 == 0 :
      plt.figure(figsize = (6,3))
      plt.subplot(121)
    elif index %2 ==1 :
      plt.subplot(122)

    plot_image(index, predictions, test_labels, test_imgs)
    if index %2 ==1:
      plt.show()
      #imgFile = 'resultImg/result{}.png'.format(index)
      #plt.savefig(imgFile)



